In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
from datetime import datetime

### datos necesarios para el dataframe:

- web scrapping: price, product_name, product_id, product_brand
- url
- fecha del dia

--> el dataframe solo podrá tener una entrada por dia y por producto (poner consiciones de si ya existe la fecha de hoy para ese nombre de producto no añadir nada)


--> columnas (url, brand, name, id, price)


### output a) un único DF:

Un dataframe con las siguientes caracteristicas:

- el dataframe solo podrá tener una entrada por cada producto al día (poner condiciones de si ya existe la fecha de hoy para ese nombre de producto no añadir nada)

- columnas (date_hyphen, date_slash, date_number, url, brand, name, id, price)

### output B) un DF por cada producto:

De esta manera cada dia sería una única entrada por cada DF

###  supuesto 1. test con una silla en la que hay stock y precio

In [2]:
url_cedric = 'https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/'

In [3]:
response_cedric = requests.get(url_cedric)
html_cedric = response_cedric.content
parsed_content_cedric = bs4.BeautifulSoup(html_cedric, "html.parser")

In [4]:
price_cedric_class = parsed_content_cedric.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0]

In [5]:
#final_price_cedric = price_cedric_class.text.replace("€","").strip().replace(",",".")
final_price_cedric = re.sub("[^\d|\,]","",price_cedric_class.text).replace(",",".")
final_price_cedric

'252.74'

In [6]:
final_name_cedric = str(parsed_content_cedric).split("ficha_product_name='")[1].split("';")[0].strip()
final_name_cedric

'Cedric B07SRYBJ6L'

In [7]:
final_id_cedric = str(parsed_content_cedric).split("ficha_product_id='")[1].split("';")[0].strip()
final_id_cedric

'B07SRYBJ6L'

In [8]:
final_brand_cedric = str(parsed_content_cedric).split("ficha_product_brand='")[1].split("';")[0].strip()
final_brand_cedric

'Cedric'

In [9]:
date_hyphen = datetime.today().strftime('%Y-%m-%d')
date_hyphen

'2022-04-16'

In [10]:
date_slash = datetime.today().strftime('%Y/%m/%d')
date_slash

'2022/04/16'

In [11]:
date_number = int(datetime.today().strftime('%Y%m%d'))
date_number

20220416

###  supuesto 2. test cuando no hay stock

In [12]:
url_no_price = "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/silla-gamer-bgeu-a136-sencillez-y-buen-precio-ofertas-2021/"

In [13]:
response_np = requests.get(url_no_price)
html_np = response_np.content
parsed_content_np = bs4.BeautifulSoup(html_np, "html.parser")
price_cedric_np = parsed_content_np.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text

In [14]:
price_cedric_np

''

In [15]:
#si está sin stock aparecerá "" --> ponerle condicion para que cuando pase sea un "0"

In [16]:
if price_cedric_np == "":
    print(0)

0


In [17]:
#hay que pasarlo al list comprenhension (una opcion sería que cuando esté el dataframe ponerle un apply(función para susitituirlo))

### supuesto 3. test cuando ya no existe el producto

In [18]:
url_wrong = "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/femor/"

In [19]:
# ¡OJO! que como tenemos un plugin al final, en este caso si que detecta un producto, y no es el que queremos, es el [0 de los bestseleers]

In [20]:
response_wrong = requests.get(url_wrong)
html_wrong = response_wrong.content
parsed_content_wrong = bs4.BeautifulSoup(html_wrong, "html.parser")
price_wrong_class = parsed_content_wrong.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip()

In [21]:
price_wrong_class

'No products found.'

##### vamos a ver si esta clase existe en los supuestos 1. y 2.

In [22]:
#nos vamos al supuesto de que hay stock y precio
price_cedric_np0 = parsed_content_cedric.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip()
price_cedric_np0

'(568)\n \n\n\n\n252,74 €\n\nVer Precio en Amazon\n\n\n\n\n\nÚltima actualización el 2022-04-16'

In [23]:
#nos vamos al supuesto de cuando no hay stock y aplicamos la clase a ver si existe
price_cedric_np2 = parsed_content_np.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip()
price_cedric_np2

'Silla Racing BGEU-A136 \n\n\n\n\n\n\n\nVer en Amazon\n\n\n\n\n\nÚltima actualización el 2021-08-09'

In [24]:
#si existe, por lo que podemos empezar con esta condicion para todos los uspuestos a la hora de obtener el precio

### OBTENCIÓN DE VARIABLES

In [46]:
urls_ergonomia = [
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/tlv-myx-801-1/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn55bk/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cashoffice-silla-ergonomica/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/noblewell/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/sihoo-lb14/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/ronda-silla-espanola/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/diablo-v-master/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/diablo-v-basic/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn61bkv1/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposapies/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn86bk/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/femor/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/umi/"   
]

In [47]:
urls_oficina = [
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obn52bk/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obn22bk/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/allguest-cedric/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/hbada-hdny108bm-eu/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/intimate-wm-heart-sillon-b07x8tqh96/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/vinsetto-sillon-de-oficina-azul-claro/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/exofcer-mc6310/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obg24b/"
]

In [48]:
urls_rodilla = [
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/duehome/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/himimi-silla-de-rodillas/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/varier/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/cinius/"
]

In [49]:
urls_gaming = [
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/gtplayer-rosa/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-gamer-2-0/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-silla-gamer-barata/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/nokaxus-yk-6008-rosa/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-nayuki/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/silla-gamer-bgeu-a136-sencillez-y-buen-precio-ofertas-2021/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/autofull-pink-bunny/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/adec-drw/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/femor-gaming/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/hbada-gaming-hdjy001bmj-cb/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/corsair-t3-rush/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/dxracer-king-ks06/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-horn/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-ray/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/dxracer-formula-f08/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-kitsune/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-takamikura/"
#"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/songmics-racing/" - no usamos
]

In [50]:
urls_products_list = urls_ergonomia + urls_oficina + urls_rodilla + urls_gaming

In [51]:
#contenido "parseado" para las siguientes variables
parsed_products_content_list = [bs4.BeautifulSoup(requests.get(i).content, "html.parser") for i in urls_products_list]

#contenido "parseado" para el precio
parsed_products_price_class = [i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text for i in parsed_products_content_list]

#PRECIO
#final_price_products = [re.sub("[^\d|\,]","",i).replace(",",".") for i in parsed_products_price_class]
final_price_products_list =[]
final_price_products_status =[]
def price_list(parsed_products_content_list,urls_products_list):
    for i,e in zip(parsed_products_content_list, range(len(urls_products_list))):
        try:
            i.find_all("div",{"class":"wp-block-media-text__content"})[0]
        except:
            if  urls_products_list[e] == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/":
                price_supuesto_0 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_0_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_0)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_0_cleaned))
                final_price_products_status.append("correcto")
            else:
                final_price_products_list.append(999999999999999)
                final_price_products_status.append("revisar")
        else:
            if i.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip() == "No products found.":
                final_price_products_list.append(0)
                final_price_products_status.append("descatalogado")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text == "":
                final_price_products_list.append(0)
                final_price_products_status.append("sin_stock")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text != "":
                price_supuesto_1 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_1_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_1)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_1_cleaned))
                final_price_products_status.append("correcto")
            else:
                print("revisar funcion price")
price_list(parsed_products_content_list,urls_products_list)

#funcionpara handling error
def handling_error_vars_product(i,text):
    try:
        str(i).split(text)[1]
    except:
        return "none"
    else:
        return str(i).split(text)[1].split("';")[0].strip()
    
#NAME
#final_name_products = [str(i).split("ficha_product_name='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_name_products = [handling_error_vars_product(i,"ficha_product_name='") for i in parsed_products_content_list]

#ID
#final_id_products = [str(i).split("ficha_product_id='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_id_products = [handling_error_vars_product(i,"ficha_product_id='") for i in parsed_products_content_list]


#BRAND
#final_brand_products = [str(i).split("ficha_product_brand='")[1].split("';")[0].strip() for i in parsed_products_content_list]
final_brand_products = [handling_error_vars_product(i,"ficha_product_brand='") for i in parsed_products_content_list]


#DATES
final_date_hyphen_products = [datetime.today().strftime('%Y-%m-%d') for i in range(len(urls_products_list))]
final_date_slash_products = [datetime.today().strftime('%Y/%m/%d') for i in range(len(urls_products_list))]
final_date_number_products = [int(datetime.today().strftime('%Y%m%d')) for i in range(len(urls_products_list))]

In [52]:
final_price_products_list

[89.99,
 131.99,
 83.94,
 252.74,
 139.99,
 229.99,
 107.54,
 489.99,
 339.99,
 234.99,
 135.99,
 95.05,
 0,
 149.99,
 219.99,
 250.99,
 134.28,
 64.99,
 236.79,
 0,
 159.99,
 128.99,
 172.24,
 136.99,
 99.9,
 112.99,
 313.84,
 147.21,
 169.99,
 179.99,
 90.09,
 259.99,
 115.04,
 0,
 242.24,
 89.9,
 0,
 189.99,
 147.05,
 279.98,
 419.0,
 254.99,
 259.99,
 349.25,
 169.95,
 189.95]

## Create the DATAFRAME

### a) un único DF:

#### paso 1: beber del csv o crear un DF vacío

In [53]:
try:
    df_single = pd.read_csv("../files/df_single.csv")
except: #si da error es que no existe, por lo que tendremos que crear el dataframe de cero
    try:
        df_single.head() 
    except: #si da error es que no existe y lo crearemos
        columns = ["date_hyphen","date_slash","date_number","product_name","product_id","product_brand","price","status","url"]
        df_single = pd.DataFrame(columns=columns)
        #df_single["date_hyphen"] = df_single["date_hyphen"].astype("datetime64")
    else: #si no da error es que existe y no haremos nada
        pass
else: #si no da error es que existe y no haremos nada
    pass

In [54]:
df_single

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url
0,2022-04-14,2022/04/14,20220414,TLV TLV-MYX-801-1,B08T1TSMQQ,TLV,89,correcto,https://sillasybienestar.com/ergonomia/sillas-...
1,2022-04-14,2022/04/14,20220414,Songmics OBN55BK,B07ZCJVFDJ,Songmics,0,sin_stock,https://sillasybienestar.com/ergonomia/sillas-...
2,2022-04-14,2022/04/14,20220414,Cashoffice B081QDV9VR,B081QDV9VR,Cashoffice,83,correcto,https://sillasybienestar.com/ergonomia/sillas-...
3,2022-04-14,2022/04/14,20220414,Cedric B07SRYBJ6L,B07SRYBJ6L,Cedric,252,correcto,https://sillasybienestar.com/ergonomia/sillas-...
4,2022-04-14,2022/04/14,20220414,Noblewell B08DR42HFV,B08DR42HFV,Noblewell,139,correcto,https://sillasybienestar.com/ergonomia/sillas-...
...,...,...,...,...,...,...,...,...,...
133,2022-04-16 00:00:00,2022/04/16,20220416,diablo X-horn,B07Z9CTB5C,diablo,254,correcto,https://sillasybienestar.com/gaming/sillas-gam...
134,2022-04-16 00:00:00,2022/04/16,20220416,diablo X-Ray,B07Q57WD7S,diablo,259,correcto,https://sillasybienestar.com/gaming/sillas-gam...
135,2022-04-16 00:00:00,2022/04/16,20220416,DXRacer Formula F08,B073GF2L1H,dxracer,349,correcto,https://sillasybienestar.com/gaming/sillas-gam...
136,2022-04-16 00:00:00,2022/04/16,20220416,Newskill Kitsune,B073TWN367,Newskill,169,correcto,https://sillasybienestar.com/gaming/sillas-gam...


#### paso 2: insertar nuevas filas en df_single - pd.concat()

In [55]:
df_append_new_files = pd.DataFrame({
    "date_hyphen":final_date_hyphen_products,
    "date_slash":final_date_slash_products,
    "date_number":final_date_number_products,
    "product_name":final_name_products,
    "product_id":final_id_products,
    "product_brand":final_brand_products,
    "price":final_price_products_list,
    "status":final_price_products_status,
    "url": urls_products_list
})
#df_append_new_files["date_hyphen"] = df_append_new_files["date_hyphen"].astype("datetime64")
df_append_new_files["price"] = df_append_new_files["price"].astype("int64")

In [56]:
if df_single.empty == True:
    df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)
elif df_append_new_files["date_hyphen"][0] == df_single["date_hyphen"][0]:
    pass
elif df_append_new_files["date_hyphen"][0] != df_single["date_hyphen"][0]:
    df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)

In [57]:
df_single.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date_hyphen    184 non-null    object
 1   date_slash     184 non-null    object
 2   date_number    184 non-null    int64 
 3   product_name   184 non-null    object
 4   product_id     184 non-null    object
 5   product_brand  184 non-null    object
 6   price          184 non-null    int64 
 7   status         184 non-null    object
 8   url            184 non-null    object
dtypes: int64(2), object(7)
memory usage: 13.1+ KB


In [58]:
df_single

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url
0,2022-04-14,2022/04/14,20220414,TLV TLV-MYX-801-1,B08T1TSMQQ,TLV,89,correcto,https://sillasybienestar.com/ergonomia/sillas-...
1,2022-04-14,2022/04/14,20220414,Songmics OBN55BK,B07ZCJVFDJ,Songmics,0,sin_stock,https://sillasybienestar.com/ergonomia/sillas-...
2,2022-04-14,2022/04/14,20220414,Cashoffice B081QDV9VR,B081QDV9VR,Cashoffice,83,correcto,https://sillasybienestar.com/ergonomia/sillas-...
3,2022-04-14,2022/04/14,20220414,Cedric B07SRYBJ6L,B07SRYBJ6L,Cedric,252,correcto,https://sillasybienestar.com/ergonomia/sillas-...
4,2022-04-14,2022/04/14,20220414,Noblewell B08DR42HFV,B08DR42HFV,Noblewell,139,correcto,https://sillasybienestar.com/ergonomia/sillas-...
...,...,...,...,...,...,...,...,...,...
179,2022-04-17,2022/04/17,20220417,diablo X-horn,B07Z9CTB5C,diablo,254,correcto,https://sillasybienestar.com/gaming/sillas-gam...
180,2022-04-17,2022/04/17,20220417,diablo X-Ray,B07Q57WD7S,diablo,259,correcto,https://sillasybienestar.com/gaming/sillas-gam...
181,2022-04-17,2022/04/17,20220417,DXRacer Formula F08,B073GF2L1H,dxracer,349,correcto,https://sillasybienestar.com/gaming/sillas-gam...
182,2022-04-17,2022/04/17,20220417,Newskill Kitsune,B073TWN367,Newskill,169,correcto,https://sillasybienestar.com/gaming/sillas-gam...


## exportar a csv

In [59]:
df_single_to_csv = df_single.to_csv("../files/df_single.csv", sep=",", index=False)

In [60]:
df_append_new_files_to_csv = df_append_new_files.to_csv("../files/df_append_new_files_to_csv_"+str(datetime.today().strftime('%Y-%m-%d'))+".csv", sep=",", index=False)

In [61]:
df_single

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url
0,2022-04-14,2022/04/14,20220414,TLV TLV-MYX-801-1,B08T1TSMQQ,TLV,89,correcto,https://sillasybienestar.com/ergonomia/sillas-...
1,2022-04-14,2022/04/14,20220414,Songmics OBN55BK,B07ZCJVFDJ,Songmics,0,sin_stock,https://sillasybienestar.com/ergonomia/sillas-...
2,2022-04-14,2022/04/14,20220414,Cashoffice B081QDV9VR,B081QDV9VR,Cashoffice,83,correcto,https://sillasybienestar.com/ergonomia/sillas-...
3,2022-04-14,2022/04/14,20220414,Cedric B07SRYBJ6L,B07SRYBJ6L,Cedric,252,correcto,https://sillasybienestar.com/ergonomia/sillas-...
4,2022-04-14,2022/04/14,20220414,Noblewell B08DR42HFV,B08DR42HFV,Noblewell,139,correcto,https://sillasybienestar.com/ergonomia/sillas-...
...,...,...,...,...,...,...,...,...,...
179,2022-04-17,2022/04/17,20220417,diablo X-horn,B07Z9CTB5C,diablo,254,correcto,https://sillasybienestar.com/gaming/sillas-gam...
180,2022-04-17,2022/04/17,20220417,diablo X-Ray,B07Q57WD7S,diablo,259,correcto,https://sillasybienestar.com/gaming/sillas-gam...
181,2022-04-17,2022/04/17,20220417,DXRacer Formula F08,B073GF2L1H,dxracer,349,correcto,https://sillasybienestar.com/gaming/sillas-gam...
182,2022-04-17,2022/04/17,20220417,Newskill Kitsune,B073TWN367,Newskill,169,correcto,https://sillasybienestar.com/gaming/sillas-gam...


In [62]:
df_append_new_files

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url
0,2022-04-17,2022/04/17,20220417,TLV TLV-MYX-801-1,B08T1TSMQQ,TLV,89,correcto,https://sillasybienestar.com/ergonomia/sillas-...
1,2022-04-17,2022/04/17,20220417,Songmics OBN55BK,B07ZCJVFDJ,Songmics,131,correcto,https://sillasybienestar.com/ergonomia/sillas-...
2,2022-04-17,2022/04/17,20220417,Cashoffice B081QDV9VR,B081QDV9VR,Cashoffice,83,correcto,https://sillasybienestar.com/ergonomia/sillas-...
3,2022-04-17,2022/04/17,20220417,Cedric B07SRYBJ6L,B07SRYBJ6L,Cedric,252,correcto,https://sillasybienestar.com/ergonomia/sillas-...
4,2022-04-17,2022/04/17,20220417,Noblewell B08DR42HFV,B08DR42HFV,Noblewell,139,correcto,https://sillasybienestar.com/ergonomia/sillas-...
5,2022-04-17,2022/04/17,20220417,sihoo lb14,B07GNDDNMW,sihoo,229,correcto,https://sillasybienestar.com/ergonomia/sillas-...
6,2022-04-17,2022/04/17,20220417,ronda la silla espanola HC1159GR,B074MJDSK6,ronda,107,correcto,https://sillasybienestar.com/ergonomia/sillas-...
7,2022-04-17,2022/04/17,20220417,diablo v-master,B079BS7DW9,diablo,489,correcto,https://sillasybienestar.com/ergonomia/sillas-...
8,2022-04-17,2022/04/17,20220417,diablo v-basic,B079BSRV7P,diablo,339,correcto,https://sillasybienestar.com/ergonomia/sillas-...
9,2022-04-17,2022/04/17,20220417,songmics OBN61BKV1,B082WQQW8Q,songmics,234,correcto,https://sillasybienestar.com/ergonomia/sillas-...
